# README
This document describes the constraint equations for cells to satisfy the hook pattern.

In [1]:
from ortools.sat.python import cp_model
model = cp_model.CpModel()
solver = cp_model.CpSolver()

N = 9

from itertools import product

Nrange = [i+1 for i in range(N)] # (1 to N)
ijrange   = list(product(*[Nrange]*2))

## Variables
* $x_{i,j,h} = 1$ $\Leftrightarrow$ cell $(i,j)$ belongs to hook $h$.
    * For each $(i,j)$, only one $(h)$ can be true, i.e. $\sum_{h} x_{i,j,h} = 1$ for all $(i,j)$.
    * For each $(h)$, exactly $2h-1$ values should be true, i.e. $\sum_{i,j} x_{i,j,h} = 2h-1$ for all $(h)$. 
        * This may appear redundant given the later constraints on the shape of the hook, but since hook $(h=1)$ doesn't really have a shape, it doesn't have a shape constraint. Thus, this constraint ensures that $1$ appears in the solution.
* $d_{i,j,h} = 1$ $\Leftrightarrow$ cells $(i,j)$ and $(i+1,j)$ both belong to hook $h$, i.e. $(i,j)$ has a neighbor when looking down.
    * Can be allowed if $2d_{i,j,h} \leq x_{i,j,h} + x_{i+1,j,h}$ for all $(i,j,h)$.
* $r_{i,j,h} = 1$ $\Leftrightarrow$ cells $(i,j)$ and $(i,j+1)$ both belong to hook $h$, i.e. $(i,j)$ has a neighbor when looking right.
    * Can be allowed if $2r_{i,j,h} \leq x_{i,j,h} + x_{i,j+1,h}$ for all $(i,j,h)$.
* $z_{i,j,h} = 1$ $\Leftrightarrow$ if the vertical leg of hook $(h)$ occupies column $(j)$, and the horizontal leg occupies row $(i)$. 
    * For each $(h)$, only one $(i,j)$ can be turned on, i.e. $\sum_{i,j} z_{i,j,h} = 1$ for all $h$. 
    * Note that for $(h=1)$, the hook technically doesn't have a vertical or horizontal leg.

In [2]:
x = {}
z = {}
d = {}
r = {}
for i,j in ijrange:
    for h in Nrange:
        x[i,j,h] = model.NewBoolVar(f'x[{i},{j},{h}]')
        z[i,j,h] = model.NewBoolVar(f'z[{i},{j},{h}]')
        if i < N:
            d[i,j,h] = model.NewBoolVar(f'd[{i},{j},{h}]')
        if j < N:
            r[i,j,h] = model.NewBoolVar(f'r[{i},{j},{h}]')

# uniqueness
for i,j in ijrange:
    model.AddExactlyOne(x[i,j,h] for h in Nrange)

for h in Nrange: 
    # each hook has size 2h-1
    model.Add(sum(x[i,j,h] for i,j in ijrange) == 2*h-1)

    # each hook has one associated (row, col)
    model.AddExactlyOne(z[i,j,h] for i,j in ijrange)

# adjacency
for h in Nrange:
    for i,j in ijrange:
        if i < N:
            model.Add(2*d[i,j,h] <= x[i,j,h] + x[i+1,j,h])
        if j < N:
            model.Add(2*r[i,j,h] <= x[i,j,h] + x[i,j+1,h])
    model.Add(sum(d[i,j,h] for i,j in ijrange if i < N) == h-1)
    model.Add(sum(r[i,j,h] for i,j in ijrange if j < N) == h-1)

## Summary of method
* To ensure all hooks are connected, it is sufficient to require exactly $h-1$ down-adjacencies and $h-1$ right-adjacencies for each hook.
    * $\sum_{i,j} d_{i,j,h} = 2h-1$ for all $h$.
    * $\sum_{i,j} r_{i,j,h} = 2h-1$ for all $h$.
    * The above assumes that one hook of each size ($1$ to $N$) exists in the $N \times N$ grid. 
* The vertical leg of a hook should occupy exactly one column, so summing $d_{i,j,h}$ over a column $j$ must give either $0$ or $h-1$. 
    This is where we use $z_{i,j,h}$ to our advantage, since $\sum_i z_{i,j,h}$ will be $1$ if column $(j)$ is used for the leg, and $0$ otherwise.
    * $\sum_{j} d_{i,j,h} = (h-1) \sum_{i} z_{i,j,h}$ for all $(i)$ for all $(h>1)$.
* The horizontal leg is treated similarly.
    * $\sum_{i} d_{i,j,h} = (h-1) \sum_{j} z_{i,j,h}$ for all $(j)$ for all $(h>1)$.


In [3]:
## HOOK constraints
for h in Nrange:
    if h == 1: # no shape constraint for hook 1
        continue
    for j in Nrange:
        model.Add(sum(d[i,j,h] for i in Nrange if i < N) <= (h-1) * sum(z[i,j,h] for i in Nrange))
    for i in Nrange:
        model.Add(sum(r[i,j,h] for j in Nrange if j < N) <= (h-1) * sum(z[i,j,h] for j in Nrange))

## Test solution

In [4]:
def assignSol(solver, N, x,z,r,d):
    import numpy as np
    X = np.zeros((N, N, N+1), dtype=int)
    H = np.zeros((N, N), dtype=int)
    R = np.zeros((N, N), dtype=int)
    D = np.zeros((N, N), dtype=int)
    Z = np.zeros((N, N), dtype=int)
    Z2 = [None for i in range(N+1)]
    for i,j in ijrange:
        for h in Nrange:
            if solver.BooleanValue(x[i,j,h]):
                # print(f"{i},{j} -> {h}")
                X[i-1,j-1,h] += 1
                H[i-1,j-1] = h
            if i < N and solver.BooleanValue(d[i,j,h]):
                D[i-1,j-1] += h
            if j < N and solver.BooleanValue(r[i,j,h]):
                R[i-1,j-1] += h
            if solver.BooleanValue(z[i,j,h]):
                Z[i-1,j-1] = h
                Z2[h] = (i,j)

    return X, H, Z, R, D

status = solver.Solve(model)
if status == cp_model.INFEASIBLE:
    print('No solution found')
else:
    X,H,Z,R,D = assignSol(solver, N, x,z,r,d)
print(H)

[[1 2 3 4 5 6 7 8 9]
 [2 2 3 4 5 6 7 8 9]
 [3 3 3 4 5 6 7 8 9]
 [4 4 4 4 5 6 7 8 9]
 [5 5 5 5 5 6 7 8 9]
 [6 6 6 6 6 6 7 8 9]
 [7 7 7 7 7 7 7 8 9]
 [8 8 8 8 8 8 8 8 9]
 [9 9 9 9 9 9 9 9 9]]
